In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from geovision.data import get_dataset_from_key, get_dataset_keys
get_dataset_keys()


['imagenette_hdf5_classification', 'imagenette_imagefolder_classification']

In [18]:
from pathlib import Path
from geovision.data.imagenette import ImagenetteImagefolderClassification

ds = ImagenetteImagefolderClassification(
    root = Path.home() / "datasets" / "imagenette",
    split = "train" 
)

display(ds)

imagenette dataset for classification
local @ [/home/sambhav/datasets/imagenette]
with 10 classes: ('cassette_player', 'chain_saw', 'church', 'english_springer', 'french_horn', 'garbage_truck', 'gas_pump', 'golf_ball', 'parachute', 'tench')
and 9376 images loaded under train split

In [46]:
import tempfile
with tempfile.NamedTemporaryFile() as tempfile:
    print(Path(tempfile.name).is_file())
    print(tempfile.name)

True
/tmp/tmpkwe_idis


In [50]:
(Path.home() / "datasets" / "imagenette").is_dir()

True